# Explingo Experiment Runner

This notebook:
1. Loads the gold-standard dataset, prepares the metrics functions, and verifies that the metric functions give the maximum score on the gold-standard dataset, and lower scores on less aligned datasets
2. Runs the prompt-design, few-shot, and bootstrap-few-shot experiments on a testing dataset

## Setup
Import necessary libraries and prepare the LLM

**Note: To run these cells, you need a `keys.yaml` file in the top-level Explingo directory with the following line:**
```yaml
openai_api_key: <your_openai_api_key>
```

In [20]:
import pandas as pd

from experiment_runner import ExplingoExperimentRunner
import os
import yaml
import dspy
import metrics
import random

In [21]:
with open(os.path.join("..", "keys.yaml"), "r") as file:
    config = yaml.safe_load(file)
    openai_api_key = config["openai_api_key"]

llm = dspy.OpenAI(model='gpt-4o', api_key=openai_api_key, max_tokens=1000)

Now, we create the main experiment runner object. This object takes in a dataset, and then
1. Splits the dataset into a training dataset and a testing dataset (see notes below)
2. Sets up the evaluation metrics (see notes below). The fluency metric is set up to use sample from the dataset as reference
3. Runs the experiments on the testing dataset

Some examples in the testing datasets include gold-standard narratives; others include only a sample explanation.
- The former makes up the gold-standard dataset used for tuning the evaluation metrics and providing few-shot examples.
- The latter makes up the testing dataset used for evaluation and for bootstrapping few-shot examples

We use the following metrics, all scored on a scale from 0-4:
- Accuracy: the narrative accurately describes the information in the explanation
- Fluency: the narrative is coherent and natural, as compared to the gold-standard explanations. We pass in a small list of sample narratives from the gold-standard dataset to compare against
- Conciseness: the narrative is not too long, as compared to the gold-standard explanations. For now, any narrative that is no longer than the longest gold-standard narrative will score 4
- Completeness: the narrative includes all relevant information from the original explanation 

**Note: You can set `verbose=1` to see the narratives generated, or `verbose=2` to see the explanations, narratives, and rationalizations**

In [22]:
# iterate all datasets in the eval_data folder
runners = {}
total_eval = 0
for dataset in os.listdir(os.path.join("eval_data")):
    runners[dataset] = ExplingoExperimentRunner(llm=llm, openai_api_key=openai_api_key, dataset_filepath = os.path.join("eval_data", dataset), verbose=1)
    total_eval += len(runners[dataset].eval_data)
    
print("Total eval examples:", total_eval)
results = []

eval_data\housing_1.json
Total number of examples: 35
Labeled training examples: 5
Labeled evaluation examples: 15
Unlabeled training examples: 5
Unlabeled evaluation examples: 10
---
eval_data\housing_2.json
Total number of examples: 22
Labeled training examples: 5
Labeled evaluation examples: 7
Unlabeled training examples: 5
Unlabeled evaluation examples: 5
---
eval_data\housing_3.json
Total number of examples: 22
Labeled training examples: 5
Labeled evaluation examples: 8
Unlabeled training examples: 5
Unlabeled evaluation examples: 4
---
eval_data\mushroom_1.json
Total number of examples: 30
Labeled training examples: 5
Labeled evaluation examples: 6
Unlabeled training examples: 5
Unlabeled evaluation examples: 14
---
eval_data\mushroom_2.json
Total number of examples: 30
Labeled training examples: 5
Labeled evaluation examples: 6
Unlabeled training examples: 5
Unlabeled evaluation examples: 14
---
eval_data\pdf_1.json
Total number of examples: 30
Labeled training examples: 5
Label

## Basic prompt design experiment

We begin with basic prompts. With 4 metrics (without completeness), each with a score of 0-2, the maximum score is 8. 

We generate narratives/rationalizations on `max_iters=5` sample explanations, and return the average total score.

In [23]:
# Utilities for cleaner results

def pretty_print(result):
    s = f"Total score: {result[0]}"
    s2 = ", ".join([f"{k}: {v}" for k, v in result[1].items()])
    print(f"{s} ({s2})")
    
def update_results(method, dataset, scores, kwargs):
    result = {"dataset": dataset, "prompt": prompt, "total score": scores[0]}
    result.update(scores[1])
    result.update(kwargs)
    results.append(result)

In [24]:
prompts = ["You are helping users understand an ML model's prediction. Given an explanation and information about the model, "
           "convert the explanation into a human-readable narrative.",
           "You are helping users who do not have experience working with ML understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Make your answers sound as natural as possible.",
           "You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Be sure to explicitly mention all values from the explanation in your response.",
]

for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for prompt in prompts:
        print(f"Prompt: {prompt}")
        scores = runner.run_basic_prompting_experiment(prompt=prompt, max_iters=2)
        update_results("basic_prompting", dataset, scores, {"prompt": prompt})
        pretty_print(scores)
        print("--")
    print("=====")

Dataset: housing_1.json
Prompt: You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative.
Narrative: The model predicts the house price based on several key factors. The house is located within the Ames city limits, specifically in the NoRidge neighborhood, which contributes significantly to the price with a value of 23069.89. The above ground living area is 2198 square feet, adding 20125.75 to the price. The second floor has 1053 square feet, contributing 18094.05. The overall material and finish of the house are rated at 8, which adds 9655.79 to the price. Lastly, the house was originally constructed in the year 2000, contributing 8192.46 to the final predicted price.
Total Score: 15.0
accuracy: 4.0, completeness: 4.0, fluency: 3.0, conciseness: 4, 
--
Narrative: The model predicts the house price based on several factors. The type of foundation being wood decreases the pri

In [25]:
llm.inspect_history(n=1)




You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Be sure to explicitly mention all values from the explanation in your response.
---
Follow the following format
Context: what the model predicts
Explanation: explanation of the model's prediction
Explanation Format: format the explanation is given in
Narrative: human-readable narrative version of the explanation
---
Context: The model predicts whether a student will pass their class
Explanation: (In a romantic relationship, yes, -1.86), (Student's guardian, father, 1.12), (Family eductional support, no, -1.02)
Explanation Format: SHAP feature contribution in (feature_name, feature_value, contribution) format
Please provide the output field Narrative. Do so immediately, without additional content before or after, and precisely as the format above shows.

Narrative: The model predicts that the student will pass their 

"\n\n\nYou are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Be sure to explicitly mention all values from the explanation in your response.\n---\nFollow the following format\nContext: what the model predicts\nExplanation: explanation of the model's prediction\nExplanation Format: format the explanation is given in\nNarrative: human-readable narrative version of the explanation\n---\nContext: The model predicts whether a student will pass their class\nExplanation: (In a romantic relationship, yes, -1.86), (Student's guardian, father, 1.12), (Family eductional support, no, -1.02)\nExplanation Format: SHAP feature contribution in (feature_name, feature_value, contribution) format\nPlease provide the output field Narrative. Do so immediately, without additional content before or after, and precisely as the format above shows.\n\n\x1bNarrative: The model predicts that the stu

## Few-shot experiment

Next, we repeat the experiment with the addition of N few-shot examples from the gold-standard dataset.

In [26]:
for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for i in [1, 3, 5]:
        print(f"Few-shot n: {i}")
        scores = runner.run_few_shot_experiment(n_few_shot=i, prompt=prompts[0], max_iters=2)
        update_results("few_shot", dataset, scores, {"n_few_shot": i, "prompt": prompts[0]})
        pretty_print(scores)
        print("--")
    print("=====")

Dataset: housing_1.json
Few-shot n: 1
Narrative: The house's location in NoRidge increased the predicted price by about $23,000. The relatively larger above ground living space increased the price by about $20,000. The house's larger than average second floor increased the price by about $18,000. The good overall material and finish of the house (rated 8/10) increased the price by about $9,600. The house is newer than average, with a construction year of 2000, which increased the price by about $8,200.
Total Score: 16.0
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 4, 
--
Narrative: The house's wood foundation reduced the predicted price by about $18,000. The house's location in Mitchel reduced the price by about $13,500. The lower-than-average material rating also reduced the house price by about $10,000. The relatively larger three season porch area increased the price by about $10,000. Having only one bedroom above ground increased the price by about $9,000.
Total Sco

In [27]:
llm.inspect_history(n=1)




You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative.
---
Follow the following format
Context: what the model predicts
Explanation: explanation of the model's prediction
Explanation Format: format the explanation is given in
Narrative: human-readable narrative version of the explanation
---
Example 1
Context: The model predicts whether a student will pass their class
Explanation: (Family eductional support, no, -1.37), (School, MS, -0.59)
Explanation Format: SHAP feature contribution in (feature_name, feature_value, contribution) format
Narrative: The lack of family support and attending the MS school indicate a lower probability of passing.
Example 2
Context: The model predicts whether a student will pass their class
Explanation: (Family eductional support, no, -2.26), (In a romantic relationship, no, 1.11), (Sex, M, -0.60)
Explanation Format: SHAP feature contributio

"\n\n\nYou are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative.\n---\nFollow the following format\nContext: what the model predicts\nExplanation: explanation of the model's prediction\nExplanation Format: format the explanation is given in\nNarrative: human-readable narrative version of the explanation\n---\nExample 1\nContext: The model predicts whether a student will pass their class\nExplanation: (Family eductional support, no, -1.37), (School, MS, -0.59)\nExplanation Format: SHAP feature contribution in (feature_name, feature_value, contribution) format\nNarrative: The lack of family support and attending the MS school indicate a lower probability of passing.\nExample 2\nContext: The model predicts whether a student will pass their class\nExplanation: (Family eductional support, no, -2.26), (In a romantic relationship, no, 1.11), (Sex, M, -0.60)\nExplanation Format: SHAP

## Bootstrapped few-shot
Next, we repeat the experiment with the addition of 3 examples bootstrapped by DSPy to optimize the evaluation metrics.

In [28]:
for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    #for i, j in [[0, 1], [0, 3], [3, 3]]:
    for i, j in [[3, 3]]:
        print(f"Few-shot n: {i}, Bootstrapped n: {j}")
        scores = runner.run_bootstrap_few_shot_experiment(n_labeled_few_shot=i, n_bootstrapped_few_shot=j, max_iters=2)
        update_results("bootstrap_few_shot", dataset, scores, {"n_few_shot": i, "n_bootstrapped_few_shot": j, "prompt": prompts[0]})
        pretty_print(scores)
        print("--")
    print("=====")

Dataset: housing_1.json
Few-shot n: 3, Bootstrapped n: 3


  0%|          | 0/10 [00:00<?, ?it/s]

Narrative: The model predicts the house price based on several key features. The above ground living area, which is 1256 square feet, decreases the predicted price by $12,527.46. The overall material and finish rating of the house, which is rated as 5, reduces the price by $10,743.76. The absence of a second floor (0 square feet) further decreases the price by $10,142.29. The house's location within the Edwards neighborhood in Ames city limits lowers the price by $9,913.81. However, the presence of a wood deck area measuring 736 square feet increases the predicted price by $9,846.38.
Total Score: 16.0
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 4, 
--


 10%|█         | 1/10 [00:00<00:03,  2.77it/s]

Narrative: The presence of a second floor with 854 square feet increases the predicted house price by about $12,000. The house being relatively new, built in 2003, adds approximately $9,000 to its price. However, the total basement area of 856 square feet reduces the price by about $6,000. Similarly, the first floor area of 856 square feet decreases the price by about $5,000. Lastly, the house's location in the CollgCr neighborhood reduces the price by about $5,000.
Total Score: 16.0
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 4, 
--


...

Bootstrapped 3 full traces after 8 examples in round 0.
Narrative: The fact that the student is in a romantic relationship and does not have family educational support suggests they are less likely to pass the class. Additionally, having their mother as their guardian slightly decreases the likelihood of passing.
Total Score: 16.0
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 4, 
--


  0%|          | 0/10 [00:00<?, ?it/s]

Narrative: The model predicts that the student is less likely to pass their class because they do not have family educational support, which negatively impacts the prediction by 2.26 units. Additionally, the fact that the student is not in a romantic relationship positively influences the prediction by 1.11 units. Lastly, the student's gender being male slightly decreases the likelihood of passing by 0.60 units.
Total Score: 16.0
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 4, 
--
Narrative: The presence of family educational support and the lack of a romantic relationship suggest the student is more likely to pass the class. However, being 17 years old slightly decreases the probability of passing.
Total Score: 16.0
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 4, 
--
Narrative: Being in a romantic relationship, the lack of family support, and the sex (male) suggest the student is less likely to pass the class.
Total Score: 14.0
accuracy: 4.0, completene

 70%|███████   | 7/10 [00:00<00:00, 114.25it/s]


Bootstrapped 3 full traces after 8 examples in round 0.
Narrative: Being in a romantic relationship and lacking family educational support suggest the student is less likely to pass the class. However, having their father as their guardian positively influences the prediction.
Total Score: 16.0
accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 4, 
--
Total score: 16.0 (accuracy: 4.0, completeness: 4.0, fluency: 4.0, conciseness: 4.0)
--
=====


In [29]:
llm.inspect_history(n=1)




You are helping users understand an ML model's prediction. Given an explanation and information about the model,
convert the explanation into a human-readable narrative.

---

Follow the following format.

Context: what the ML model predicts

Explanation: explanation of an ML model's prediction

Explanation Format: format the explanation is given in

Narrative: human-readable narrative version of the explanation

---

Context: The model predicts whether a student will pass their class

Explanation: (Family eductional support, no, -2.26), (In a romantic relationship, no, 1.11), (Sex, M, -0.60)

Explanation Format: SHAP feature contribution in (feature_name, feature_value, contribution) format

Narrative: The model predicts that the student is less likely to pass their class because they do not have family educational support, which negatively impacts the prediction by 2.26 units. Additionally, the fact that the student is not in a romantic relationship positively influences the predi

"\n\n\nYou are helping users understand an ML model's prediction. Given an explanation and information about the model,\nconvert the explanation into a human-readable narrative.\n\n---\n\nFollow the following format.\n\nContext: what the ML model predicts\n\nExplanation: explanation of an ML model's prediction\n\nExplanation Format: format the explanation is given in\n\nNarrative: human-readable narrative version of the explanation\n\n---\n\nContext: The model predicts whether a student will pass their class\n\nExplanation: (Family eductional support, no, -2.26), (In a romantic relationship, no, 1.11), (Sex, M, -0.60)\n\nExplanation Format: SHAP feature contribution in (feature_name, feature_value, contribution) format\n\nNarrative: The model predicts that the student is less likely to pass their class because they do not have family educational support, which negatively impacts the prediction by 2.26 units. Additionally, the fact that the student is not in a romantic relationship posi

In [30]:
result_df = pd.DataFrame(results)
result_df.to_csv("results.csv")
result_df

,dataset,prompt,total score,accuracy,completeness,fluency,conciseness,n_few_shot,n_bootstrapped_few_shot
0,housing_1.json,You are helping users understand an ML model's...,15.5,4.0,4.0,3.5,4.0,NaN,NaN
1,housing_1.json,You are helping users who do not have experien...,14.5,4.0,4.0,2.5,4.0,NaN,NaN
2,housing_1.json,You are helping users understand an ML model's...,15.5,4.0,4.0,3.5,4.0,NaN,NaN
3,housing_2.json,You are helping users understand an ML model's...,14.5,4.0,4.0,2.5,4.0,NaN,NaN
4,housing_2.json,You are helping users who do not have experien...,14.0,4.0,4.0,2.0,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
58,mushroom_2.json,You are helping users understand an ML model's...,10.5,0.0,3.0,3.5,4.0,3.0,3.0
59,pdf_1.json,You are helping users understand an ML model's...,15.0,4.0,3.0,4.0,4.0,3.0,3.0
60,pdf_2.json,You are helping users understand an ML model's...,9.5,0.0,2.0,3.5,4.0,3.0,3.0
61,student_1.json,You are helping users understand an ML model's...,16.0,4.0,4.0,4.0,4.0,3.0,3.0
